In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path().cwd().parent)
print(Path().cwd())

/home/das/QuantUS_Projs/QuantUS-Plugins/CLI-Demos
/home/das/QuantUS_Projs/QuantUS-Plugins


## Select Image & Phantom and Parse

In [19]:
from quantus.image_loading.utc_loaders.options import get_scan_loaders

print("Available scan types:", list(get_scan_loaders().keys()))

Available scan types: ['philips_pkl', 'philips_rf_sc_3d', 'canon_iq', 'clarius_rf', 'siemens_rf', 'terason_rf']


In [20]:
scan_type = 'canon_iq'

scan_path = '/home/das/QuantUS_Projs/Test Data/Canon Data/Canon Fat study - sorted/006/Preset 1/20220606090949_IQ.bin'
phantom_path = '/home/das/QuantUS_Projs/Test Data/Canon Data/Canon Fat study - sorted/Phantom data/Preset_1/20220831121844_IQ.bin'
scan_loader_kwargs = {}

In [21]:
from quantus.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, phantom_path, **scan_loader_kwargs)

## Load Segmentation

In [7]:
from quantus.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['pkl_roi', 'nifti_voi']


In [8]:
seg_loader = 'pkl_roi'

seg_path = '/home/das/QuantUS_Projs/Test Data/Canon Data/ROIs/pkl_files/006/20220606090949_IQ/whole.pkl'
seg_loader_kwargs = {
    'assert_scan': False, # Checks if the seg is initially from the same scan
    'assert_phantom': False, # Checks if the seg is initially from the same phantom
}

In [9]:
from quantus.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_loader, image_data, seg_path, scan_path, phantom_path, **seg_loader_kwargs)

## Load Analysis Configuration

In [10]:
from quantus.analysis_config.utc_config.options import get_config_loaders

print("Available analysis config loaders:", list(get_config_loaders().keys()))

Available analysis config loaders: ['pkl_rf', 'philips_3d_config', 'clarius_L15_config', 'clarius_C3_config', 'custom']


In [11]:
config_loader = 'custom'

config_path = '/home/das/QuantUS_Projs/Test Data/Canon Data/Canon Fat study - sorted/Phantom data/analysis.pkl'
config_kwargs = {
  'transducer_freq_band': [0, 8000000], # Frequency band of the transducer in Hz
  'analysis_freq_band': [3000000, 5000000], # Frequency band for analysis in Hz
  'center_frequency': 4000000, # Center frequency of the transducer in Hz
  'sampling_frequency': 53330000, # Sampling frequency of the scan in Hz
  'ax_win_size': 3.85, # Axial window size in mm
  'lat_win_size': 3.85, # Lateral window size in mm
  'axial_overlap': 0.75, # Axial window overlap in percent
  'lateral_overlap': 0.75, # Lateral window overlap in percent
  'window_thresh': 0.95 # Percentage of window area required to be considered in ROI
}

In [12]:
from quantus.entrypoints import analysis_config_step

config_data = analysis_config_step(config_loader, config_path, scan_path, phantom_path, **config_kwargs)

## Customize & Perform Analysis

In [13]:
from quantus.analysis.options import get_analysis_types, get_required_kwargs

all_analysis_types, all_analysis_funcs = get_analysis_types()
print("Available analysis types:", list(all_analysis_types.keys()))

Available analysis types: ['paramap']


In [14]:
analysis_type = 'paramap'

print("Available analysis functions:", list(all_analysis_funcs[analysis_type].keys()))

Available analysis functions: ['compute_power_spectra', 'lizzi_feleppa', 'attenuation_coef', 'bsc', 'nakagami_params', 'hscan']


In [15]:
analysis_funcs = ['lizzi_feleppa', 'attenuation_coef', 'bsc', 'nakagami_params'] # list of analysis functions to run (empty list means all)

# Find all required kwargs
analysis_funcs = analysis_funcs if len(analysis_funcs) else list(all_analysis_funcs[analysis_type].keys())
required_kwargs = get_required_kwargs(analysis_type, analysis_funcs)
print("Required kwargs for current analysis:", required_kwargs)

Required kwargs for current analysis: ['ref_bsc', 'ref_attenuation']


In [16]:
analysis_kwargs = {
    'ref_attenuation': 0.7, # dB/cm/MHz
    'ref_bsc': 0.0006, # 1/cm-sr (@ center frequency)
}

In [17]:
from quantus.entrypoints import analysis_step

analysis_data = analysis_step(analysis_type, image_data, config_data, seg_data, analysis_funcs, **analysis_kwargs)

100%|██████████| 1106/1106 [00:17<00:00, 63.39it/s]


## Customize & Perform Visualizations

In [19]:
from quantus.visualizations.options import get_visualization_types

all_visualization_types, all_visualization_funcs = get_visualization_types()

print("Available visualization types:", list(all_visualization_types.keys()))

Available visualization types: ['paramap']


In [20]:
visualization_type = 'paramap'

print("Available visualization functions:", list(all_visualization_funcs[visualization_type].keys()))

Available visualization functions: ['plot_ps_window_data', 'plot_hscan_result', 'plot_hscan_wavelets', 'paramaps']


In [22]:
# List of visualization functions to run (empty list means no visualization)
visualization_funcs = ['paramaps', 'plot_ps_window_data'] 

visualization_kwargs = {
    # 'paramap_folder_path': 'visualization_results', # folder to save visualizations
    'hide_all_visualizations': True,
}

In [23]:
import matplotlib.pyplot as plt
from quantus.entrypoints import visualization_step

visualization_data = visualization_step(visualization_type, analysis_data, visualization_funcs, **visualization_kwargs)

plt.close('all')

## Customize & Perform Data Export

In [24]:
from quantus.data_export.options import get_data_export_types

all_data_export_types, all_data_export_funcs = get_data_export_types()
print("Available data export types:", list(all_data_export_types.keys()))

Available data export types: ['csv', 'pkl']


In [25]:
data_export_type = 'csv'

print("Available data export functions:", list(all_data_export_funcs[data_export_type].keys()))

Available data export functions: ['descr_vals', 'hscan_stats', 'radiomics_stats']


In [32]:
data_export_funcs = ['descr_vals', 'radiomics_stats']
data_export_path = 'test.csv'
data_export_kwargs = {
    'pyradiomics_yaml_paths': [
        '/home/das/QuantUS_Projs/QuantUS-Plugins/configs/pyradiomics/moderate.yml',
        '/home/das/QuantUS_Projs/QuantUS-Plugins/configs/pyradiomics/coarse.yml',
        '/home/das/QuantUS_Projs/QuantUS-Plugins/configs/pyradiomics/fine.yml',
        ]
}

In [33]:
from quantus.entrypoints import data_export_step

data_export_obj = data_export_step(data_export_type, visualization_data, data_export_path, data_export_funcs, **data_export_kwargs)